In [1]:
import argparse
import copy
import glob
import numpy as np
from molSimplifyAD.dbclass_mongo import tmcMongo
from molSimplifyAD.utils.pymongo_tools import connect2db, insert, count_find, convert2dataframe
from molSimplifyAD.ga_tools import get_mulliken

Using TensorFlow backend.


## Connect to db

In [2]:
from os.path import expanduser
home = expanduser("~")
dbconfig = json.load(open(home + "/.db_config"))

db = connect2db(user=dbconfig['user'], pwd=dbconfig['pwd'],
                host="localhost", port=27017,
                database='tmc', auth=True)

In [3]:
doc = db.oct.find_one({"tag": "OxoHAT"})

In [4]:
doc['outpath']

u'/home/nandy/cr_full_enumeration/geo_outfiles/gen_0/gen_0_slot_2_0_3_1_15_31_20_2.out'

## Query db for documents needed for update

In [ ]:
constraints = {"alpha": {"$in": [20.0]},
#                "lig6": {"$ne": "x"},
               "tag": "OxoHAT",
               "converged": True
               }
cursor = db.oct.find(constraints,
                     no_cursor_timeout=True).batch_size(10)

tot = count_find(cursor)
print(tot)
cursor = db.oct.find(constraints,
                     no_cursor_timeout=True).batch_size(10)

In [ ]:
# df = convert2dataframe(db, collection='oct', constraints=constraints, normalized=True)
# df.to_csv("test.csv", index=False)

## Update document (Here an example of recalculate ss_flag and update this value)

In [ ]:
failed_list = []
update_fields = ['ss_flag'] # fields for update need to be in this list
count = 0
converted = 0
for _tmcdoc in cursor:
    recovered = True
    if (np.isnan(_tmcdoc['ss_flag']) or _tmcdoc['ss_flag']==-1) and  _tmcdoc['converged']:
        try:
            ## Convert a db document to tmcMongo class object with DFTrun.
            _this_tmc = tmcMongo(document=_tmcdoc, tag=_tmcdoc["tag"], subtag=_tmcdoc["subtag"],
                                 publication=_tmcdoc["publication"], update_fields=update_fields)
        except:
            recovered = False
        if recovered:
            ###-----Here is your customized block of code-----
            if _this_tmc.this_run.converged:
                if _this_tmc.this_run.spin == 1:
                    _this_tmc.document['ss_flag'] = 1
                else:
                    if abs(_this_tmc.this_run.ss_target -_this_tmc.this_run.ss_act) < 1:
                         _this_tmc.document['ss_flag'] = 1
                    else:
                         _this_tmc.document['ss_flag'] = 0
            ###-----Here is your customized block of code-----
            __ = insert(db, "oct", _this_tmc, debug=False)
            converted += 1
        else:
            print("Failed on recovering: ", _tmcdoc["tag"], _tmcdoc["unique_name"])
            failed_list.append(_tmcdoc["unique_name"])
    count += 1
    if count % 100 == 0:
        print("processed %d, converted %d, total %d" % (count, converted, tot))

## Update JP's jpca-redox

In [ ]:
from molSimplify.job_manager.tools import textfile
from molSimplify.Classes.mol3D import mol3D

In [ ]:
def geo_flag_from_doc(doc):
    mol_init = mol3D()
    mol_init.readfromtxt(doc['init_geo'].split('\n'))
    mol_opt = mol3D()
    mol_opt.readfromtxt(doc['opt_geo'].split('\n'))
    flag_oct, flag_list, dict_oct_info = mol_opt.IsOct(debug=False)
    return flag_oct

In [ ]:
constraints = {"alpha": {"$in": [20.0]},
               "tag": "jpca-redox",
               "converged": True
               }
base_outpath = '/home/crduan/jpca-redox/'

In [ ]:
failed_list = []
update_fields = ['ss_flag', 'ss_act', 'ss_target', "geo_flag", 'ligstr']  # fields for update need to be in this list
count = 0
converted = 0
for _tmcdoc in cursor:
    recovered = True
    if (np.isnan(_tmcdoc['ss_flag']) or _tmcdoc['ss_flag'] == -1) and _tmcdoc['converged']:
        try:
            # Convert a db document to tmcMongo class object with DFTrun.
            _this_tmc = tmcMongo(document=_tmcdoc, tag=_tmcdoc["tag"], subtag=_tmcdoc["subtag"],
                                 publication=_tmcdoc["publication"], update_fields=update_fields)
        except:
            recovered = False
        if recovered:
            # -----Here is your customized block of code-----
            if _this_tmc.this_run.converged:
                base_outpath = '/home/crduan/jpca-redox/'
                outfile = base_outpath + _tmcdoc['outpath'].split("/")[-1]
                output = textfile(outfile)
                spin = int(output.wordgrab(['Spin multiplicity:'], -1)[0][0])
                energy, ss_act, ss_target, tot_time, thermo_grad_error, solvent_cont, tot_step = output.wordgrab(
                    ['FINAL', 'S-SQUARED:', 'S-SQUARED:', 'processing', 'Maximum component of gradient is too large',
                     'C-PCM contribution to final energy:', 'Optimization Cycle'],
                    [2, 2, 4, 3, 0, 4, 3], last_line=True)
                if spin == 1:
                    ss_target = 0
                    ss_act = 0
                    ss_flag = 1
                else:
                    try:
                        ss_target = float(ss_target.strip(')'))
                        ss_act = float(ss_act)
                        ss_flag = 1 if abs(ss_target-ss_act) < 1 else 0
                    except:
                        ss_target = np.nan
                        ss_act = np.nan
                        ss_flag = np.nan
                geo_flag = geo_flag_from_doc(_tmcdoc)
                _this_tmc.document['ss_flag'] = ss_flag
                _this_tmc.document['ss_act'] = ss_act
                _this_tmc.document['ss_target'] = ss_target
                _this_tmc.document['geo_flag'] = geo_flag
                _this_tmc.document['ligstr'] = "_".join(_this_tmc.document['liglist'])
                _this_tmc.document['unique_name'] = _tmcdoc['unique_name']
                print(_this_tmc.document['unique_name'], geo_flag, ss_flag, ss_act, ss_target)
            # -----Here is your customized block of code-----
            __ = insert(db, "oct", _this_tmc, debug=False)
            converted += 1
        else:
            print("Failed on recovering: ",
                  _tmcdoc["tag"], _tmcdoc["unique_name"])
            failed_list.append(_tmcdoc["unique_name"])
    count += 1
    if count % 100 == 0:
        print("processed %d, converted %d, total %d" % (count, converted, tot))